In [1]:
import os
import sys
import cv2
import zmq
import time
import math
import json
import carla
import pickle
import random
import numpy as np
from threading import Lock
sys.path.append(r'/home/avcem/CarlaUE4/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [2]:
path = "dataset_autopilot"
os.makedirs(path, exist_ok=True)

In [3]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
traffic_manager = client.get_trafficmanager()
blueprint = world.get_blueprint_library()

In [4]:
# Destroying old actors i.e old sensors and vehicles
# for x in world.get_actors().filter('*sensor*'):
#     x.destroy()
# for x in world.get_actors().filter('*vehicle*'):
#     x.destroy()

# Blueprint of the vehicles and sensors
vehicle_bp = blueprint.find('vehicle.volkswagen.t2_2021')
camera_bp = blueprint.find('sensor.camera.rgb')
imu_bp = blueprint.find('sensor.other.imu')
gnss_bp = blueprint.find('sensor.other.gnss')

In [5]:
spawn_points = world.get_map().get_spawn_points()

In [6]:
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
image = None
camera_transform = carla.Transform(carla.Location(z=2.5, x=0.65))
gnss_transform = carla.Transform(carla.Location(z=2.0))
imu_transform = carla.Transform(carla.Location(z=2.0))
gnss = None
imu = None
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [7]:
def euler_to_quaternion():
    pitch, roll = 0, 0
    yaw=vehicle.get_transform().rotation.yaw
    cy = math.cos(yaw * 0.5)
    sy = math.sin(yaw * 0.5)
    cp = math.cos(pitch * 0.5)
    sp = math.sin(pitch * 0.5)
    cr = math.cos(roll * 0.5)
    sr = math.sin(roll * 0.5)

    return {
        "w": cr * cp * cy + sr * sp * sy,
        "x": sr * cp * cy - cr * sp * sy,
        "y": cr * sp * cy + sr * cp * sy,
        "z": cr * cp * sy - sr * sp * cy
    }

# Callbacks
def image_callback(data):
    global image
    array = np.frombuffer(data.raw_data, dtype=np.uint8)
    array = np.reshape(array, (data.height, data.width, 4))[:, :, :3]
    image = array
def gnss_callback(event):
    global gnss
    gnss = {
        "latitude": event.latitude,
        "longitude": event.longitude
    }
    
def imu_callback(event):
    global imu
    imu = {
        "orientation": euler_to_quaternion(),
        "orientation_covariance": [0.0]*9,  
        "angular_velocity": {
            "x": event.gyroscope.x,
            "y": event.gyroscope.y,
            "z": event.gyroscope.z
        },
        "angular_velocity_covariance": [0.0]*9,  
        "linear_acceleration": {
            "x": event.accelerometer.x,
            "y": event.accelerometer.y,
            "z": event.accelerometer.z
        },
        "linear_acceleration_covariance": [0.0]*9 
    }

In [8]:
imu_callbackvehicle = world.try_spawn_actor(vehicle_bp,random.choice(spawn_points))
camera_sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
gnss_sensor = world.spawn_actor(gnss_bp, gnss_transform, attach_to=vehicle)
imu_sensor = world.spawn_actor(imu_bp, imu_transform, attach_to=vehicle)

In [9]:
# traffic_manager.ignore_lights_percentage(vehicle, 100)
# traffic_manager.ignore_signs_percentage(vehicle, 100)
# traffic_manager.ignore_walkers_percentage(vehicle, 100)
# traffic_manager.ignore_vehicles_percentage(vehicle, 100)

In [10]:
camera_sensor.listen(lambda data: image_callback(data))
gnss_sensor.listen(lambda data: gnss_callback(data))
imu_sensor.listen(lambda data: imu_callback(data))

In [11]:
frame = {'count': 0}
data_log = []

In [ ]:
def send_data(image):
    img_name = f"{frame['count']:05g}.png"
    cv2.imwrite(os.path.join(path, img_name), image)

    control = vehicle.get_control()
    throttle = control.throttle
    steer = control.steer
    brake = control.brake

    data_log.append({
        'img': img_name,
        'steer': steer,
        'throttle': throttle,
        'brake': brake,
        'imu': imu,
        'gnss': gnss
    })

    frame['count'] += 1
    if frame['count'] >= 10000:
        camera_sensor.stop()

In [ ]:
# for i in range(10000):
#     try:
#         save_data(image)
#         time.sleep(0.1)
#     finally:
#         with open(os.path.join(path, "logs.json"), "w") as f:
#             json.dump(data_log, f, indent=2)
#             print("Done logging")

In [17]:
vehicle.set_autopilot(True)

while True:
    cv2.imshow("Camera", image)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
port = 6969
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind(f"tcp://*:{port}")

In [ ]:
try:
    payload = {
            image, steer
        }
        self.socket.send(pickle.dumps(payload))
except KeyboardInterrupt:
    print("Interrupted by user")
except Exception as e:
    print(f"Fatal Error: {e}")
finally:
    try:
        socket.unbind(f"tcp://*:{port}")
        print("Socket unbound.")
    except zmq.ZMQError as e:
        print(f"ZMQ unbind error: {e}")
        socket.close()
        context.term()


In [ ]:
print(vehicle.get_control())

In [ ]:
for ac in world.get_actors().filter('*vehicle*'):
    # ac.destroy()
    print(ac)